In [3]:
import numpy as np 
import pandas as pd
import re

# Untuk split data
from sklearn.model_selection import train_test_split

# Untuk Algoritma ML yang akan di pakai
from sklearn.neural_network import MLPClassifier

# untuk tuning hyperparameter
from sklearn.model_selection import GridSearchCV

# untuk stopwords
from nltk.corpus import stopwords

# untuk membangun pipeline ML
from sklearn.pipeline import Pipeline

# bagian dari pipe line untuk handling kolom 
from sklearn.compose import ColumnTransformer

# menghitung nilai f1
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, classification_report

from sklearn.feature_extraction.text import CountVectorizer

# menyimpan model
import pickle

In [7]:
data = pd.read_csv('train_preprocess.tsv.txt', sep='\t',header = None)
data

,0,1
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative
...,...,...
10995,tidak kecewa,positive
10996,enak rasa masakan nya apalagi kepiting yang me...,positive
10997,hormati partai-partai yang telah berkoalisi,neutral
10998,"pagi pagi di tol pasteur sudah macet parah , b...",negative


In [9]:
df = data.rename(columns={0: 'Text', 1: 'Sentimen'})
df.head()

,Text,Sentimen
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative


# Normalisasi Data

In [12]:
def cleansing(sent):
    # Mengubah kata menjadi huruf kecil semua dengan menggunakan fungsi lower()
    string = sent.lower()
    # Menghapus emoticon dan tanda baca menggunakan "RegEx" dengan script di bawah
    string = re.sub(r'[^a-zA-Z0-9]', ' ', string)
    return string

In [14]:
df['text_clean'] = df.Text.apply(cleansing)

In [16]:
df.head()

,Text,Sentimen,text_clean
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive,warung ini dimiliki oleh pengusaha pabrik tahu...
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral,mohon ulama lurus dan k212 mmbri hujjah partai...
2,lokasi strategis di jalan sumatera bandung . t...,positive,lokasi strategis di jalan sumatera bandung t...
3,betapa bahagia nya diri ini saat unboxing pake...,positive,betapa bahagia nya diri ini saat unboxing pake...
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative,duh jadi mahasiswa jangan sombong dong kas...


# Stopwords NLTK

In [19]:
# Mengambil stopwords bahasa Indonesia
stop_words = set(stopwords.words('indonesian'))

In [21]:
# Contoh teks
text_list = df.text_clean.tolist()
clean_text_list = []

In [23]:
for text in text_list:
    words = text.split()
    clean_words = [word for word in words if word.lower() not in stop_words]
    clean_text = ' '.join(clean_words)
    clean_text_list.append(clean_text)

In [25]:
df['text_filter'] = pd.Series(clean_text_list)
df

,Text,Sentimen,text_clean,text_filter
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive,warung ini dimiliki oleh pengusaha pabrik tahu...,warung dimiliki pengusaha pabrik puluhan terke...
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral,mohon ulama lurus dan k212 mmbri hujjah partai...,mohon ulama lurus k212 mmbri hujjah partai diw...
2,lokasi strategis di jalan sumatera bandung . t...,positive,lokasi strategis di jalan sumatera bandung t...,lokasi strategis jalan sumatera bandung nya ny...
3,betapa bahagia nya diri ini saat unboxing pake...,positive,betapa bahagia nya diri ini saat unboxing pake...,betapa bahagia nya unboxing paket barang nya b...
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative,duh jadi mahasiswa jangan sombong dong kas...,duh mahasiswa sombong kasih kartu kuning belaj...
...,...,...,...,...
10995,tidak kecewa,positive,tidak kecewa,kecewa
10996,enak rasa masakan nya apalagi kepiting yang me...,positive,enak rasa masakan nya apalagi kepiting yang me...,enak masakan nya kepiting menyenangkan memilih...
10997,hormati partai-partai yang telah berkoalisi,neutral,hormati partai partai yang telah berkoalisi,hormati partai partai berkoalisi
10998,"pagi pagi di tol pasteur sudah macet parah , b...",negative,pagi pagi di tol pasteur sudah macet parah b...,pagi pagi tol pasteur macet parah bikin jengkel


# Feature Extraction

In [28]:
# df_vektor = df.text_filter.tolist()
df_vektor = df.text_filter.tolist()

In [30]:
count_vect = CountVectorizer()
    
# melakukan fitting dan transformasi pada dokumen
count_vect.fit(df_vektor)

# melihat hasil representasi bag of words
X = count_vect.fit_transform(df_vektor)

In [32]:
pickle.dump(count_vect, open("feature_New.sav", "wb"))

# Train -Test Data

In [35]:
y = df.Sentimen

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 40)

In [39]:
model = Pipeline([ ('algoritma', MLPClassifier()) ])
parameter_grid = {
    'algoritma__hidden_layer_sizes': [{i} for i in [1, 10]],
    'algoritma__activation': ['relu','tanh','logistic'],
    'algoritma__learning_rate_init' : [0.01],
    'algoritma__alpha': [0.1,0.01,1],
    'algoritma__early_stopping': [True]
} 

model_NN = GridSearchCV(model, parameter_grid)

In [41]:
%%time
model_NN.fit(X_train,y_train)

CPU times: total: 7min 11s
Wall time: 7min 17s


GridSearchCV(estimator=Pipeline(steps=[('algoritma', MLPClassifier())]),
             param_grid={'algoritma__activation': ['relu', 'tanh', 'logistic'],
                         'algoritma__alpha': [0.1, 0.01, 1],
                         'algoritma__early_stopping': [True],
                         'algoritma__hidden_layer_sizes': [{1}, {10}],
                         'algoritma__learning_rate_init': [0.01]})

In [43]:
model_NN.best_params_ 

{'algoritma__activation': 'tanh',
 'algoritma__alpha': 0.01,
 'algoritma__early_stopping': True,
 'algoritma__hidden_layer_sizes': {10},
 'algoritma__learning_rate_init': 0.01}

In [45]:
pickle.dump(model_NN, open("model_NN.sav", 'wb'))

# Evaluasi Model

For all Data

In [49]:
y_pred = model_NN.predict(X)

In [51]:
print(classification_report(y,y_pred))

              precision    recall  f1-score   support

    negative       0.90      0.90      0.90      3436
     neutral       0.95      0.87      0.91      1148
    positive       0.94      0.96      0.95      6416

    accuracy                           0.93     11000
   macro avg       0.93      0.91      0.92     11000
weighted avg       0.93      0.93      0.93     11000



For Data Training

In [54]:
y_pred_train = model_NN.predict(X_train)

In [56]:
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

    negative       0.93      0.93      0.93      2775
     neutral       0.98      0.92      0.95       915
    positive       0.96      0.97      0.96      5110

    accuracy                           0.95      8800
   macro avg       0.96      0.94      0.95      8800
weighted avg       0.95      0.95      0.95      8800



For Data Testing

In [60]:
y_pred_test = model_NN.predict(X_test)

In [62]:
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

    negative       0.78      0.79      0.78       661
     neutral       0.81      0.65      0.72       233
    positive       0.90      0.92      0.91      1306

    accuracy                           0.85      2200
   macro avg       0.83      0.79      0.80      2200
weighted avg       0.85      0.85      0.85      2200

